# Text classification with TensorFlow Hub: Movie reviews

This notebook classifies movie reviews as positive or negative using the text of the review. This is an example of binary classification.

For this example we will use `IMDB dataset` that contains the text of 50,000 movie reviews from the Internet Movie Database. These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.

In [1]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.8.0
GPU is available


#### Download the IMDB dataset

In [2]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", 
                                                   split=('train[:60%]', 'train[60%:]', 'test'),
                                                   as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTWBQSH/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTWBQSH/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTWBQSH/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


#### Explore the data

Each example is a sentence representing the movie review and a corresponding label. The sentence is not preprocessed in any way. The label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

In [9]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print("Train example batch: ", train_examples_batch[0])
print()
print("Train labels batch: ", train_labels_batch[0])

Train example batch:  tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)

Train labels batch:  tf.Tensor(0, shape=(), dtype=int64)


#### Build the model

One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages:

  * we don't have to worry about text preprocessing,
  * we can benefit from transfer learning,
  * the embedding has a fixed size, so it's simpler to process.
  
For this example we will use a pre-trained text embedding model from TensorFlow Hub called `google/tf2-preview/gnews-swivel-20dim/1`.

Other three available embeddings that could be used in this tutorial are:

  * `google/tf2-preview/gnews-swivel-20dim-with-oov/1`- vocabulary contains OOV (Out of vocabulary) buckets. This can help if vocabulary of the task and vocabulary of the model don't fully overlap.
  * `google/tf2-preview/nnlm-en-dim50/1` - A much larger model with ~1M vocabulary size and 50 dimensions.
  * `google/tf2-preview/nnlm-en-dim128/1` - Even larger model with ~1M vocabulary size and 128 dimensions.
 
The output shape of the embeddings is `(num_examples, embedding_dimension)`

In [10]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, 
                           input_shape=[],
                           dtype=tf.string,
                           trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=469, shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

#### Build the entire model

In [11]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


#### Loss function and optimizer

A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the `binary_crossentropy` loss function.

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

#### Train the model

In [14]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 112ms/step - loss: 0.7313 - accuracy: 0.5173 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 2s 74ms/step - loss: 0.6409 - accuracy: 0.6065 - val_loss: 0.6073 - val_accuracy: 0.6333
Epoch 3/20
30/30 [==============================] - 2s 75ms/step - loss: 0.5842 - accuracy: 0.6653 - val_loss: 0.5628 - val_accuracy: 0.6879
Epoch 4/20
30/30 [==============================] - 2s 74ms/step - loss: 0.5402 - accuracy: 0.7077 - val_loss: 0.5248 - val_accuracy: 0.7160
Epoch 5/20
30/30 [==============================] - 2s 74ms/step - loss: 0.4983 - accuracy: 0.7430 - val_loss: 0.4902 - val_accuracy: 0.7539
Epoch 6/20
30/30 [==============================] - 2s 74ms/step - loss: 0.4616 - accuracy: 0.7726 - val_loss: 0.4581 - val_accuracy: 0.7787
Epoch 7/20
30/30 [==============================] - 2s 73ms/step - loss: 0.4238 - accuracy: 0.8023 - val_loss: 0.4290 - val_accuracy: 0.7879
Epoc

#### Evaluate the model

In [15]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3158 - accuracy: 0.8598
loss: 0.316
accuracy: 0.860
